I'm going to experiment with the embeddings -- i think that some improvement could be seen by adding/swapping to the POS embeddings, which should represent the sentence structure more accurately. 

In [33]:
import pandas as pd
from sklearn.calibration import LabelEncoder
import numpy as np
# 1. Load the data
df = pd.read_csv('./megablunders.csv')

to keep performance manageable, let's just stick with 10-20 example sentences   

In [34]:
df.head()

,original_sentence,error
0,By dropping a game to the pathetic Tampa Bay D...,DM
1,"Although, if history is any indication, the te...",FRAG
2,"The Sox almost never go down uneventfully, whi...",PR
3,Because of the accumulated bad karma that hang...,AGREE
4,The team not only has squandered huge leads bu...,PAR


In [28]:
# cool, let's do some embeddings
from transformers import AutoTokenizer, AutoModel
import torch

# Load pre-trained model
model_name = "roberta-base"  # or other suitable models
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_contextual_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Use [CLS] token embedding or pooled output
    return outputs.last_hidden_state[:,0,:].numpy()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
get_contextual_embeddings(df["original_sentence"][0])

array([[-7.29245245e-02,  8.25554281e-02,  1.37330443e-02,
        -1.01086691e-01,  1.30480260e-01, -3.93946394e-02,
         2.29111835e-02,  1.54930726e-02,  4.22989205e-02,
        -1.30321249e-01, -5.19741997e-02, -2.21938193e-02,
         1.83993671e-02, -3.24645415e-02,  2.27112249e-02,
         6.57488033e-03, -8.27172548e-02, -1.82002336e-02,
        -3.08600739e-02,  9.13068652e-06, -1.33268178e-01,
         4.26415801e-02,  1.30661279e-02,  1.37425959e-01,
        -6.15940616e-02,  8.86302143e-02,  5.33035547e-02,
         8.71367604e-02, -5.40796667e-02,  4.58926819e-02,
        -6.02459237e-02, -5.80055639e-02,  8.69648904e-02,
         2.20408514e-02,  3.67776081e-02,  1.07054606e-01,
        -1.17216259e-02, -2.76569724e-02, -5.35630845e-02,
         5.77873178e-03,  2.66451947e-02,  1.82461649e-01,
        -4.81216609e-02, -4.17346209e-02,  6.98180273e-02,
        -3.60586494e-02,  1.62251666e-03,  1.62541475e-02,
        -3.23037580e-02, -2.29281560e-02, -1.41842179e-0

good poc, let's see how it performs in the model model 

In [36]:
embeddings = df["error"].apply(lambda x: get_contextual_embeddings(x))

In [37]:
df["error"].value_counts()

error
DM       22
PR       22
PAR      22
ROS      22
FRAG     20
MM       20
CASE     19
NONE     19
AGREE    18
C         1
Name: count, dtype: int64

In [18]:
label_encoder = LabelEncoder()
df['error'] = label_encoder.fit_transform(df['error'])

In [24]:
df["error"].value_counts()

error
10    22
9     22
11    22
6     20
7     20
5     19
4     19
8     19
2     17
1      3
3      1
0      1
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['error'], test_size=0.1, random_state=42, stratify=df["error"])

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.